# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is `Sentiment140.csv.zip` and is located  [here](https://drive.google.com/open?id=1yFnoviM3Gjt34lW4FgTKT7hnoW5WbGMz), download it, unzip it and place it in the data folder.
Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [2]:
# your code here
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import random


def clean_up(s):    
    """
    Cleans up numbers, URLs, and special characters from a string.
    
    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # Step 1 - Remove all digits
    s_1 = re.sub('\d', '', s)
    # Step 2 - Remove http:// refererences
    s_2 = re.sub('(http|https)://([^\s]+)', '', s_1)
    # Step 3 - Remove all special chars
    s_3 = re.sub('\W', ' ', s_2 )
    # Step 4 - to lower and split to list
    s_4 = (s_3.lower())
    return s_4


def tokenize(s):    
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)


def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    result = []
    for w in l:
        result.append(lemmatizer.lemmatize(ps.stem(w)))
    return result


def remove_stopwords(l):
    
    """
    Remove English stopwords from a list of strings.
    
    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words('english') 
    return [word for word in l if word not in stop_words]

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv('./data/Sentiment140.csv')

In [5]:
data.shape

(1600000, 6)

In [6]:
# split data to allow faster load times. split on the middle to have both sentiments (target)
data_sample = data[790000:810000]
data_sample.shape

(20000, 6)

In [7]:
data['target'].value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [8]:
data_sample['target'].value_counts()

4    10000
0    10000
Name: target, dtype: int64

In [9]:
data_sample.head()

,target,id,date,flag,user,text
790000,0,2325540224,Thu Jun 25 05:38:16 PDT 2009,NO_QUERY,Kain292,Virpi's last day. Busy as tits.
790001,0,2325540629,Thu Jun 25 05:38:19 PDT 2009,NO_QUERY,dean_b,@SherilynMoon unfortunate then that I have no ...
790002,0,2325540730,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,volleygalore,just dropped off the car to get the brakes loo...
790003,0,2325540825,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,shOrtyyxx,Colin Kelly on Clyde1 Should get One Republic ...
790004,0,2325540910,Thu Jun 25 05:38:21 PDT 2009,NO_QUERY,DLPsYks,just a little upset i had a chance to take 4 o...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like [this](https://drive.google.com/open?id=1ygk0Je3ViMhnyojQqz1BqdjFjwnnzzXd)

In [10]:
# your code here
data_sample['text_processed'] = data_sample.apply(lambda x: clean_up(x['text']), axis=1)
data_sample['text_processed'] = data_sample.apply(lambda x: tokenize(x['text_processed']), axis=1)
data_sample['text_processed'] = data_sample.apply(lambda x: stem_and_lemmatize(x['text_processed']), axis=1)
data_sample['text_processed'] = data_sample.apply(lambda x: remove_stopwords(x['text_processed']), axis=1)

In [12]:
data_sample.head()

,target,id,date,flag,user,text,text_processed
790000,0,2325540224,Thu Jun 25 05:38:16 PDT 2009,NO_QUERY,Kain292,Virpi's last day. Busy as tits.,"[virpi, last, day, busi, tit]"
790001,0,2325540629,Thu Jun 25 05:38:19 PDT 2009,NO_QUERY,dean_b,@SherilynMoon unfortunate then that I have no ...,"[sherilynmoon, unfortun, signal, tweet, photo,..."
790002,0,2325540730,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,volleygalore,just dropped off the car to get the brakes loo...,"[drop, car, get, brake, look, hope, thi, cost,..."
790003,0,2325540825,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,shOrtyyxx,Colin Kelly on Clyde1 Should get One Republic ...,"[colin, kelli, clyde, get, one, republ, neyo, ..."
790004,0,2325540910,Thu Jun 25 05:38:21 PDT 2009,NO_QUERY,DLPsYks,just a little upset i had a chance to take 4 o...,"[littl, upset, chanc, take, realli, cool, pict..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [13]:
# your code here
from nltk.probability import FreqDist
# Iterating over each list on text processed to extract all the words
words = [w for lst in data_sample['text_processed'] for w in lst]
# Passing the previous variable to FreqDist function
fdist = FreqDist(words)
# Using the .most_common function to extract the top 5000 words
# After running it, modify this value to 100 to be able to run it
top5000 = fdist.most_common(100)
# The previous variable returns a tuple containing the word and the freq, we are now extracting all the words
top_words = [w[0] for w in top5000]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [14]:
# your code here
#Creating final list
final_data = []
#iterrows returns index + row 
#Sliced the dataframe otherwise is heavy to run
for i, r in data_sample[8000:12000].iterrows():
    #One dict per row
    result = {}
    # Iterating over the top5000 words and then checking if the word in word500 is in the column 'text_processed'
    for w in top_words:
        result[w] = (w in r['text_processed'])
    # If the value on the column target for each row is 0 means it is negative, otherwise (4) is positive
    if r['target'] == 0:
        # Negative
        sentiment = False 
    else:
        # positive
        sentiment = True
    #Saving per each row the dict and the sentiment
    final_data.append((result, sentiment))

In [15]:
final_data[0]

({'go': False,
  'day': False,
  'work': True,
  'get': False,
  'good': False,
  'thi': True,
  'wa': False,
  'today': False,
  'like': False,
  'quot': False,
  'time': False,
  'love': False,
  'got': False,
  'one': False,
  'u': False,
  'want': False,
  'thank': False,
  'know': False,
  'think': False,
  'miss': False,
  'back': False,
  'morn': False,
  'feel': False,
  'lol': False,
  'realli': False,
  'need': False,
  'ha': False,
  'new': False,
  'hope': False,
  'well': False,
  'look': False,
  'see': False,
  'make': False,
  'amp': False,
  'twitter': False,
  'night': False,
  'still': False,
  'sad': False,
  'im': False,
  'farrah': False,
  'last': False,
  'oh': False,
  'much': False,
  'come': False,
  'home': False,
  'na': False,
  'wish': False,
  'great': False,
  'watch': False,
  'sleep': False,
  'tri': False,
  'whi': False,
  'onli': False,
  'week': True,
  'veri': False,
  'wait': False,
  'fawcett': False,
  'say': False,
  'haha': False,
  'take': 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

In [16]:
print(len(final_data))

4000


In [17]:
# your code here
# Shuffle list
random.shuffle(final_data)

# Split 80/20
training_data = final_data[:3200]
test_data = final_data[3200:]

In [18]:
classifier = nltk.NaiveBayesClassifier.train(training_data)

In [19]:
classifier.show_most_informative_features(10)

Most Informative Features
                     sad = True            False : True   =     31.1 : 1.0
                     die = True            False : True   =     17.1 : 1.0
                  follow = True             True : False  =      8.0 : 1.0
                   thank = True             True : False  =      6.3 : 1.0
                    nice = True             True : False  =      5.5 : 1.0
                    miss = True            False : True   =      5.4 : 1.0
                    hate = True            False : True   =      5.4 : 1.0
                     bed = True             True : False  =      4.6 : 1.0
                      ok = True             True : False  =      4.2 : 1.0
                   night = True             True : False  =      3.9 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [22]:
# your code here
print("Accuracy score:", nltk.classify.accuracy(classifier, test_data))

Accuracy score: 0.7325
